<h1> Classification using TimeMixer </h1>

<div>
<img src="img/time_mixer.png" width="1000"/>
</div>

<h2>1. Imports and load data</h2>

In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import DataLoader, TensorDataset


In [15]:
class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        #del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


<h2>2. Create input and target data </h2>

We use the six sensors 'eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3' 

In [16]:
df_list = ['eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3']
input_df = pd.concat([data[i] for i in df_list], axis = 1)
input_df

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2409.6,...,2.336,2.391,2.375,2.297,2.328,2.383,2.328,2.250,2.250,2.211
1,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,...,2.297,2.266,2.266,2.219,2.211,2.266,2.273,2.211,2.195,2.219
2,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2395.8,...,2.359,2.391,2.391,2.375,2.375,2.375,2.305,2.305,2.320,2.266
3,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2382.8,2382.8,...,2.117,2.219,2.281,2.227,2.164,2.164,2.219,2.250,2.273,2.273
4,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2373.0,...,2.141,2.172,2.187,2.227,2.219,2.211,2.242,2.219,2.227,2.297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,...,2.328,2.305,2.328,2.359,2.375,2.281,2.242,2.250,2.266,2.273
2201,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,...,2.273,2.383,2.359,2.297,2.297,2.336,2.406,2.461,2.461,2.406
2202,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.227,2.242,2.219,2.211,2.273,2.273,2.250,2.219,2.219,2.250
2203,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.328,2.328,2.328,2.281,2.266,2.305,2.281,2.250,2.242,2.281


Standardise the input and target data

In [17]:
# Standardise the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_target)

# Standatdise the input
scaler = StandardScaler()
input_data_scaled = scaler.fit_transform(input_df)

<h2>3. Create the model, train it & make predictions </h2>

<ul>
<li>With the TimeMixer Module we create a neural network with 1 input layer, 2 hidden layers and 1 output layer --> 4 layers total
<li>We use the stochastic gradient descent with a middle-sized batch size of 32 since we dont have a very big data set
<li>We use CrossEntropyLoss() for calculcating the loss. It uses the softmax function. Because of that, we don't have to use the softmax function in our output layer for classification by using the probabilities
</ul>

In [18]:
states = [27, 6728, 49122]

accs = []

for RANDOM_STATE in states:

    X_train, X_test, y_train, y_test = train_test_split(input_data_scaled, y_encoded, test_size=0.2, random_state=RANDOM_STATE, stratify=y_encoded)

    # Create tensors for pytorch
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # with tensors create train and test dataset
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    test_data = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

    # implementing the timemixer model
    class TimeMixer(nn.Module):
        def __init__(self, input_size, output_size, hidden_size=128, num_layers=4, dropout=0.2):
            super(TimeMixer, self).__init__()
            
            # create 1 input layer, 2 hidden layers and 1 output layer 
            self.fc1 = nn.Linear(input_size, hidden_size)
            self.fc2 = nn.Linear(hidden_size, hidden_size)
            self.fc3 = nn.Linear(hidden_size, hidden_size)
            self.fc4 = nn.Linear(hidden_size, output_size)
            self.dropout = nn.Dropout(dropout)
            
            # create the activation function
            self.relu = nn.ReLU()
        
        # forward pass
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.fc4(x)
            return x

    # defining the model
    model = TimeMixer(input_size=X_train.shape[1], output_size=4)  # Output size = Anzahl Klassen 
    
    # calculating the loss and optimizer
    criterion = nn.CrossEntropyLoss()  # lossfunction for classification (CrossEntropyLoss is using softmax, which is why dont use softmax in the ouptut layer)
    optimizer = optim.Adam(model.parameters(), lr=0.002)

    # training the model, using 100 epochs for each random state
    num_epochs = 100
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1) # using the scheduler to adapt the learning rate dynamically during training
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0 
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
            # Forward Pass
            outputs = model(inputs)
            
            # Calculate the loss with the loss function
            loss = criterion(outputs, targets)
            
            # Backward Pass and optimzing weights and biases
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        scheduler.step()

    # Store preds and targets for evaluation with classification report
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            
            preds = torch.argmax(outputs, axis=1)  # The max. probability defines the predicted class
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    # print the classificiation report
    print(f"Classification Report for random state {RANDOM_STATE}:")
    print(classification_report(all_targets, all_preds, zero_division=0.0))

    # Calcualte accuracy and append it to the list to calculate mean and std later
    accuracy = accuracy_score(all_targets, all_preds)
    accs.append(accuracy)

accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Accuracy: {accs_mean}")
print(f"Std Accuracy: {accs_std}")

Classification Report for random state 27:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        72
           1       1.00      0.99      0.99        72
           2       1.00      0.99      0.99        72
           3       1.00      1.00      1.00       225

    accuracy                           0.99       441
   macro avg       0.99      0.99      0.99       441
weighted avg       0.99      0.99      0.99       441

Classification Report for random state 6728:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        72
           1       0.99      1.00      0.99        72
           2       0.97      0.96      0.97        72
           3       0.99      1.00      0.99       225

    accuracy                           0.99       441
   macro avg       0.99      0.99      0.99       441
weighted avg       0.99      0.99      0.99       441

Classification Report for random state 491